# Activity Coefficients

Simulating how NRTL activity coefficients for the ethanol-water system evolve with temperature using PathSim blocks and connections.

The NRTL (Non-Random Two-Liquid) model computes liquid-phase activity coefficients $\gamma_i$ that quantify deviations from ideal mixing:

$$y_i P = x_i \gamma_i P_i^{\text{sat}}(T)$$

Here we wire an NRTL block and two Antoine blocks into a simulation to see how activity coefficients and the resulting bubble pressures change with temperature.

In [ ]:
import matplotlib.pyplot as plt

from pathsim import Simulation, Connection
from pathsim.blocks import Source, Scope, Function

from pathsim_chem.thermodynamics import NRTL, Antoine

## System Definition

Set up the blocks: a temperature source, Antoine correlations for both components, and an NRTL activity coefficient model for a 40/60 ethanol-water mixture.

In [ ]:
# Temperature ramp from 330 K to 380 K
T_src = Source(func=lambda t: 330 + t * 0.5)

# Antoine vapor pressure correlations (ln form, T in K, P in Pa)
antoine_ethanol = Antoine(a0=23.5807, a1=3673.81, a2=-46.681)
antoine_water = Antoine(a0=23.2256, a1=3835.18, a2=-45.343)

# NRTL activity coefficients for ethanol(1)-water(2)
nrtl = NRTL(
    x=[0.4, 0.6],
    a=[[0, -0.801], [3.458, 0]],
    b=[[0, 200], [-100, 0]],
    c=[[0, 0.3], [0.3, 0]],
)

A custom `Function` block computes the bubble pressure from the activity coefficients and pure-component vapor pressures:

$$P_{\text{bubble}} = x_1 \gamma_1 P_1^{\text{sat}} + x_2 \gamma_2 P_2^{\text{sat}}$$

In [ ]:
# Compute bubble pressure from gamma and Psat values
x1, x2 = 0.4, 0.6

bubble = Function(
    func=lambda g1, g2, P1, P2: x1 * g1 * P1 + x2 * g2 * P2
)

# Scopes to record activity coefficients and bubble pressure
scp_gamma = Scope(labels=["gamma_ethanol", "gamma_water"])
scp_bubble = Scope(labels=["P_bubble"])

## Wiring and Simulation

Connect the temperature source to NRTL and both Antoine blocks. The NRTL outputs and Antoine outputs feed into the bubble pressure calculation. Scopes record everything.

In [ ]:
sim = Simulation(
    blocks=[T_src, nrtl, antoine_ethanol, antoine_water, bubble, scp_gamma, scp_bubble],
    connections=[
        # Temperature drives all thermodynamic blocks
        Connection(T_src, nrtl, antoine_ethanol, antoine_water),
        # NRTL outputs -> scope and bubble pressure
        Connection(nrtl, scp_gamma, bubble),
        Connection(nrtl[1], scp_gamma[1], bubble[1]),
        # Antoine outputs -> bubble pressure
        Connection(antoine_ethanol, bubble[2]),
        Connection(antoine_water, bubble[3]),
        # Bubble pressure -> scope
        Connection(bubble, scp_bubble),
    ],
    dt=0.5,
)

sim.run(100)

In [ ]:
time_g, gamma = scp_gamma.read()
time_b, P_bub = scp_bubble.read()
T_range = 330 + time_g * 0.5

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.plot(T_range - 273.15, gamma[0], label=r"$\gamma_{\mathrm{ethanol}}$")
ax1.plot(T_range - 273.15, gamma[1], label=r"$\gamma_{\mathrm{water}}$")
ax1.axhline(1.0, color="gray", linestyle="--", alpha=0.5)
ax1.set_xlabel("Temperature [°C]")
ax1.set_ylabel(r"$\gamma_i$")
ax1.set_title("NRTL Activity Coefficients")
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(T_range - 273.15, P_bub[0] / 1e3)
ax2.axhline(101.325, color="gray", linestyle="-.", alpha=0.5, label="1 atm")
ax2.set_xlabel("Temperature [°C]")
ax2.set_ylabel("Bubble Pressure [kPa]")
ax2.set_title("Bubble Pressure (40/60 Ethanol-Water)")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

Both activity coefficients exceed 1, confirming positive deviation from Raoult's law — a characteristic of the ethanol-water system. The bubble pressure rises steeply with temperature, crossing 1 atm near the expected boiling point of the mixture.